In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1517936636274_0004,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2665722c

In [9]:
%%sql
DROP TABLE airports

In [10]:
%%sql
CREATE EXTERNAL TABLE airports (iata string, airport string, city string, state string, country string, llat string, llong string)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '/HdiSamples/Demo'
TBLPROPERTIES ("hive.input.dir.recursive" = "TRUE", 
    "hive.mapred.supports.subdirectories" = "TRUE",
    "hive.supports.subdirectories" = "TRUE", 
    "mapred.input.dir.recursive" = "TRUE")

In [39]:
%%sql
DROP TABLE airports_partitioned_hive

In [ ]:
%%sql
DROP TABLE airports_partitioned_hive

In [81]:
%%sh
cat /Demo2

cat: /Demo2: No such file or directory


In [49]:
%%sql
CREATE EXTERNAL TABLE airports_partitioned_hive (iata string, airport string, city string, state string, llat string, llong string)
PARTITIONED BY (country string)
STORED AS PARQUET
LOCATION '/Demo2'
TBLPROPERTIES ("hive.input.dir.recursive" = "TRUE", 
    "hive.mapred.supports.subdirectories" = "TRUE",
    "hive.supports.subdirectories" = "TRUE", 
    "mapred.input.dir.recursive" = "TRUE")

In [50]:
%%sql
ALTER TABLE airports_partitioned_hive RECOVER PARTITIONS

In [44]:
val df = spark.sql ("SELECT iata, airport, city, state, regexp_replace(country, '\"', '') AS country, llat, llong FROM airports")

df: org.apache.spark.sql.DataFrame = [iata: string, airport: string ... 5 more fields]

In [79]:
df.count()

res98: Long = 3377

In [47]:
df.write.partitionBy("country").mode("overwrite").format("parquet").option("path", "/Demo2").saveAsTable("Airports_Partitioned")

In [48]:
spark.sql("SELECT * FROM Airports_Partitioned").count()

res50: Long = 3377

In [ ]:
import org.apache.spark.sql.hive._

val hc = new HiveContext(spark.sparkContext)

In [77]:
%%sql
ALTER TABLE airports_partitioned_hive DROP PARTITION (country='USA')

In [78]:
%%sql
ALTER TABLE airports_partitioned_hive ADD PARTITION (country='USA') LOCATION '/Demo2/country=USA'

In [ ]:
hc.sql("SELECT * FROM Airports_Partitioned").count()

In [72]:
<!-- Returns 3377 misses subdirectory -->
spark.sql ("SELECT * FROM airports_partitioned_hive WHERE country='USA'").count()

res85: Long = 3363

In [76]:
import org.apache.spark.sql.hive._

val hc = new HiveContext(spark.sparkContext)

<!-- Returns 3377 misses subdirectory -->
hc.sql ("SELECT * FROM airports_partitioned_hive").count()

res95: Long = 14